## **Movielens-1M using FMClassifier and Meta data** 
Compare the prediction using FMClassifier from Spark

#### **Data Loading and Processing**

In [1]:
import pandas as pd 
import numpy as np
import pyspark.pandas as ps

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType 

from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import FMClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml import Pipeline

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-memory 192g --executor-memory 16g pyspark-shell'
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'

/home/dev/anaconda3/envs/rec/lib/python3.8/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
from pyspark import SparkContext

builder = SparkSession.builder
builder = builder.config("spark.driver.maxResultSize", "5G")

spark = builder.master("local[*]").appName("FMClassifier_MovieLens").getOrCreate()
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

24/02/09 09:25:19 WARN Utils: Your hostname, pc resolves to a loopback address: 127.0.1.1; using 192.168.18.5 instead (on interface enp4s0)
24/02/09 09:25:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/09 09:25:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Faster processing using pandas vs spark
rating = pd.read_csv("ml-1m/ratings.dat", names=["user_id", "item_id", "rating", "timestamp"], delimiter="::", engine="python")
rating.columns = ['user_id','item_id','rating','timestamp']
rating

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [4]:
rating.drop(['timestamp'], axis=1, inplace=True)

# set all positive interactions to 1
df_classification = rating.copy()
df_classification['interaction'] = 1
df_classification.drop(['rating'], axis=1, inplace=True)

In [5]:
user_features = pd.read_csv("ml-1m/users.dat", names=["user_id", "gender", "age", "occupation", "zipcode"], delimiter="::", engine="python")
user_features = user_features.drop('zipcode', axis=1)
user_features.occupation = user_features.occupation.astype('str')
user_features.age = user_features.age.astype('str')
user_features = pd.get_dummies(user_features)

user_features.head()

,user_id,gender_F,gender_M,age_1,age_18,age_25,age_35,age_45,age_50,age_56,...,occupation_19,occupation_2,occupation_20,occupation_3,occupation_4,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9
0,1,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,3,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
4,5,False,True,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [6]:
import collections

movie_desc = pd.read_csv("ml-1m/movies.dat", names=["item_id", "title", "genres"], delimiter="::", engine="python", encoding="ISO-8859-1")

# Convert genres to lowercase
movie_desc.genres = movie_desc.genres.str.lower()

split_series = movie_desc.genres.str.split('|').apply(lambda x: x)
split_series_dict = split_series.apply(collections.Counter)

multi_hot = pd.DataFrame.from_records(split_series_dict).fillna(value=0).astype('int')

item_features = pd.concat([movie_desc.item_id, multi_hot], axis=1)
item_features

,item_id,animation,children's,comedy,adventure,fantasy,romance,drama,action,crime,thriller,horror,sci-fi,documentary,war,musical,mystery,film-noir,western
0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,3949,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3880,3950,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3881,3951,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [7]:
item_names = movie_desc
item_names.head()

,item_id,title,genres
0,1,Toy Story (1995),animation|children's|comedy
1,2,Jumanji (1995),adventure|children's|fantasy
2,3,Grumpier Old Men (1995),comedy|romance
3,4,Waiting to Exhale (1995),comedy|drama
4,5,Father of the Bride Part II (1995),comedy


#### **Balancing Approach: Randomized vs PNS (Popularity-based Negative Sampling) Approach** 
For classification, since all rating converted into interaction = 1, we will try to balance with negative interaction. There are two approaches, first we can use randomized negative sampling. Take unseen interaction from space and sampling. Another approach is to see the popularized items and sampling based on users pairs that not interact with it. Assumption, users already seen the popular items but decided not to interact with it or not interested

In [8]:
# Randomized negative sampling

import random

all_users = df_classification.user_id.unique()
all_items = df_classification.item_id.unique()

negative_instances = []

# take all the items combination where user not yet interacted with
for user in all_users:
    user_interacted_item = df_classification[df_classification.user_id == user]['item_id'].unique()
    non_interacted_items = set(all_items) - set(user_interacted_item)
    for item in non_interacted_items:
        negative_instances.append([user, item, 0])

num_negatives = len(rating[df_classification['interaction'] == 1])

# randomized with the same number of positive instances
sampled_negatives = random.sample(negative_instances, num_negatives)

df_negatives = pd.DataFrame(sampled_negatives, columns=['user_id', 'item_id', 'interaction'])
df_negatives

columns = ['user_id', 'item_id', 'interaction']
balanced_df_randomized = pd.concat([df_classification[columns], df_negatives[columns]]).reset_index(drop=True)

In [9]:
# Popularity based negative sampling

popular_item = df_classification.value_counts('item_id').reset_index()
popular_item.columns = ['item_id', 'popularity']

popular_item['sampling_probability'] = popular_item['popularity'] / popular_item['popularity'].sum()
popular_item_neg_samples = popular_item.sample(n=num_negatives, weights='sampling_probability', replace=True)
popular_item_neg_samples

,item_id,popularity,sampling_probability
782,1135,407,0.000407
797,2857,399,0.000399
317,2064,798,0.000798
358,1077,744,0.000744
150,1673,1128,0.001128
...,...,...,...
161,3160,1113,0.001113
377,529,720,0.000720
16,1197,2318,0.002318
1267,3783,235,0.000235


In [10]:
negative_samples = [] 
np.random.seed(42)

users = np.random.choice(all_users, size=num_negatives)
items = popular_item_neg_samples.sample(n=num_negatives, weights='sampling_probability').item_id.values

negative_samples_df = pd.DataFrame({'user_id': users, 'item_id': items, 'interaction': np.zeros(num_negatives, dtype=int)})

# remove duplicates pair from negative samples that match in positive samples
df_classification_subset = df_classification[['user_id', 'item_id']]
is_in_positive = negative_samples_df[['user_id', 'item_id']].isin(df_classification_subset)
negative_samples_df = negative_samples_df[~is_in_positive.any(axis=1)]

# # merging
merged_df = pd.merge(negative_samples_df, df_classification_subset, on=['user_id', 'item_id'], how='left', indicator=True)
negative_samples_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')

balanced_df_pns = pd.concat([df_classification[columns], negative_samples_df[columns]]).reset_index(drop=True)
balanced_df_pns.value_counts('interaction')

interaction
1    1000209
0     864749
Name: count, dtype: int64

In [11]:
# merged meta data with the balanced dataset
df_randomized_final = pd.merge(balanced_df_randomized, user_features, on="user_id", how="left")
df_randomized_final = pd.merge(df_randomized_final, item_features, on="item_id", how="left")

df_pns_final = pd.merge(balanced_df_pns, user_features, on="user_id", how="left")
df_pns_final = pd.merge(df_pns_final, item_features, on="item_id", how="left")

In [12]:
feature_columns_set = list(df_randomized_final.columns)
feature_columns_set.remove('interaction')
print(feature_columns_set)

['user_id', 'item_id', 'gender_F', 'gender_M', 'age_1', 'age_18', 'age_25', 'age_35', 'age_45', 'age_50', 'age_56', 'occupation_0', 'occupation_1', 'occupation_10', 'occupation_11', 'occupation_12', 'occupation_13', 'occupation_14', 'occupation_15', 'occupation_16', 'occupation_17', 'occupation_18', 'occupation_19', 'occupation_2', 'occupation_20', 'occupation_3', 'occupation_4', 'occupation_5', 'occupation_6', 'occupation_7', 'occupation_8', 'occupation_9', 'animation', "children's", 'comedy', 'adventure', 'fantasy', 'romance', 'drama', 'action', 'crime', 'thriller', 'horror', 'sci-fi', 'documentary', 'war', 'musical', 'mystery', 'film-noir', 'western']


#### **Training and Eval use Randomized Negative Sampling** 

In [13]:
from pyspark.ml.feature import VectorAssembler

spark_dataframe = spark.createDataFrame(df_randomized_final)

assembler = VectorAssembler(inputCols=feature_columns_set, outputCol="features")
balanced_df_spark = assembler.transform(spark_dataframe)

balanced_df_spark.show()

24/02/09 09:25:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/02/09 09:25:44 WARN TaskSetManager: Stage 0 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


+-------+-------+-----------+--------+--------+-----+------+------+------+------+------+------+------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+------------+------------+------------+------------+------------+------------+------------+---------+----------+------+---------+-------+-------+-----+------+-----+--------+------+------+-----------+---+-------+-------+---------+-------+--------------------+
|user_id|item_id|interaction|gender_F|gender_M|age_1|age_18|age_25|age_35|age_45|age_50|age_56|occupation_0|occupation_1|occupation_10|occupation_11|occupation_12|occupation_13|occupation_14|occupation_15|occupation_16|occupation_17|occupation_18|occupation_19|occupation_2|occupation_20|occupation_3|occupation_4|occupation_5|occupation_6|occupation_7|occupation_8|occupation_9|animation|children's|comedy|adventure|fantasy|romance|drama|action|crime|t

In [14]:
(train_data, test_data) = balanced_df_spark.randomSplit([0.75, 0.25], seed=42)

# Ensure 'user_id' is present
print("Training Data Columns: ", train_data.columns)
print("Test Data Columns: ", test_data.columns)

labelIndexer = StringIndexer(inputCol="interaction", outputCol="indexedLabel").fit(balanced_df_spark)

fm = FMClassifier(featuresCol="features", labelCol="indexedLabel", stepSize=0.001)
pipeline = Pipeline(stages=[labelIndexer, fm])
model = pipeline.fit(train_data)

Training Data Columns:  ['user_id', 'item_id', 'interaction', 'gender_F', 'gender_M', 'age_1', 'age_18', 'age_25', 'age_35', 'age_45', 'age_50', 'age_56', 'occupation_0', 'occupation_1', 'occupation_10', 'occupation_11', 'occupation_12', 'occupation_13', 'occupation_14', 'occupation_15', 'occupation_16', 'occupation_17', 'occupation_18', 'occupation_19', 'occupation_2', 'occupation_20', 'occupation_3', 'occupation_4', 'occupation_5', 'occupation_6', 'occupation_7', 'occupation_8', 'occupation_9', 'animation', "children's", 'comedy', 'adventure', 'fantasy', 'romance', 'drama', 'action', 'crime', 'thriller', 'horror', 'sci-fi', 'documentary', 'war', 'musical', 'mystery', 'film-noir', 'western', 'features']
Test Data Columns:  ['user_id', 'item_id', 'interaction', 'gender_F', 'gender_M', 'age_1', 'age_18', 'age_25', 'age_35', 'age_45', 'age_50', 'age_56', 'occupation_0', 'occupation_1', 'occupation_10', 'occupation_11', 'occupation_12', 'occupation_13', 'occupation_14', 'occupation_15', '

24/02/09 09:25:45 WARN TaskSetManager: Stage 1 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:25:48 WARN TaskSetManager: Stage 4 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:25:54 WARN TaskSetManager: Stage 5 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:25:56 WARN TaskSetManager: Stage 7 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:25:57 WARN TaskSetManager: Stage 9 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:25:58 WARN TaskSetManager: Stage 11 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:25:59 WARN TaskSetManager: Stage 13 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09

In [15]:
predictions = model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="interaction", rawPredictionCol="prediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f"Area under ROC: {roc_auc}")

evaluator = BinaryClassificationEvaluator(labelCol="interaction")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

evaluator = MulticlassClassificationEvaluator(labelCol="interaction", predictionCol="prediction")

precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

print("Weighted Precision: {:.3f}".format(precision))
print("Weighted Recall: {:.3f}".format(recall))

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

24/02/09 09:27:27 WARN TaskSetManager: Stage 207 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:27:33 WARN TaskSetManager: Stage 218 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


Area under ROC: 0.5027697215527439


Accuracy: 0.4967591865201846


24/02/09 09:27:40 WARN TaskSetManager: Stage 229 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:27:45 WARN TaskSetManager: Stage 231 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


Weighted Precision: 0.609
Weighted Recall: 0.503
+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       0.0|         1.0|(50,[0,1,2,4,13,3...|
|       0.0|         1.0|(50,[0,1,2,4,13,3...|
|       0.0|         1.0|(50,[0,1,2,4,13,3...|
|       0.0|         1.0|(50,[0,1,2,4,13,3...|
|       0.0|         1.0|(50,[0,1,2,4,13,3...|
+----------+------------+--------------------+
only showing top 5 rows



24/02/09 09:27:47 WARN TaskSetManager: Stage 233 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


In [16]:
from sklearn.metrics import confusion_matrix

y_pred = predictions.select("prediction").collect()
y_orig = predictions.select("interaction").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix: \n", cm)

24/02/09 09:27:47 WARN TaskSetManager: Stage 234 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:27:53 WARN TaskSetManager: Stage 235 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


Confusion Matrix: 
 [[  2280 247216]
 [   899 248894]]


In [17]:
k = 10

def pivot_table_recommendation(recommendation):
    # Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
    pv_rec = recommendation.pivot(index='user_id', columns='rank', values='item_id').reset_index()

    # Set user_id as the index
    pv_rec.set_index('user_id', inplace=True)

    # Remove the 'rank' column
    pv_rec.columns.name = None
    pv_rec.columns = [f'{int(rank)}' for rank in pv_rec.columns]
    pv_rec.index.name = None

    return pv_rec

In [18]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# # Rank the predictions
windowSpec = Window.partitionBy("user_id").orderBy(predictions["prediction"].desc())
df_test = predictions.withColumn("rank", row_number().over(windowSpec))

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test.filter(df_test['rank'] <= 10)

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = top_10_recommendations.groupBy("user_id").pivot("rank").agg({"item_id": "first"})
pivot_recommendations = pivot_recommendations.na.fill(0)
pivot_recommendations.show()


24/02/09 09:28:04 WARN TaskSetManager: Stage 236 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:28:10 WARN TaskSetManager: Stage 249 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


+-------+----+----+----+----+----+----+----+----+----+----+
|user_id|   1|   2|   3|   4|   5|   6|   7|   8|   9|  10|
+-------+----+----+----+----+----+----+----+----+----+----+
|      7|1196|3107|3753| 783|2012|3442|  95|1379|2105| 169|
|     19|  76| 260| 368| 377| 648|1127|1197|1240|1275|1371|
|     22|  47|  95| 104| 163| 256| 368| 454| 474| 543| 589|
|     25| 110| 157| 223| 260| 546| 737|1129|1356|1371|1372|
|     26|   1|  39|  45| 104| 125| 160| 168| 195| 198| 234|
|     29|  50| 288| 589| 858| 969|1356|1374|1376|1610|2000|
|     31|1077|1304|2000|3104|3836|  48|2642|1573|2093| 861|
|     32|  50| 247| 457| 589| 608|1343|1683|2571|2916|2959|
|     34| 339| 353| 441| 455| 497| 837|1079|1100|1148|1196|
|     39| 223| 785|1060|1127|1193|2605|2706|3005|3578|3745|
|     43|2369|2581|2605|2961| 586|  12| 346|2535|2078|3622|
|     50|2241|3409|3565|3594|3786|3444| 603|2169|  50|2477|
|     54|1235|1269|2788|2477|1242|2381|3254| 367|3799| 181|
|     57|  70| 441| 661|1196|1199|1221|1

In [19]:
pivot = pivot_recommendations.toPandas().set_index('user_id')
pivot = pivot.rename_axis(None, axis=0)
pivot

24/02/09 09:28:16 WARN TaskSetManager: Stage 252 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


,1,2,3,4,5,6,7,8,9,10
7,1196,3107,3753,783,2012,3442,95,1379,2105,169
19,76,260,368,377,648,1127,1197,1240,1275,1371
22,47,95,104,163,256,368,454,474,543,589
25,110,157,223,260,546,737,1129,1356,1371,1372
26,1,39,45,104,125,160,168,195,198,234
...,...,...,...,...,...,...,...,...,...,...
6034,320,527,1248,1617,2186,3435,2674,3625,882,635
6035,1,62,92,252,256,329,355,378,420,437
6038,232,920,1079,1183,1210,1419,3088,778,1459,2688
6039,199,471,661,858,900,901,904,912,916,924


In [20]:
k = 10

test_users_items = df_test.toPandas().groupby('user_id')['item_id'].apply(set).to_dict()
comm_user = pivot.index.values

def hit_rate(val_pivot, val_test_user_items, val_comm_user):
    hit_rate = np.mean([int(len(set(val_pivot.loc[u]) & val_test_user_items[u]) > 0) for u in val_comm_user])
    return hit_rate

def reciprocal_rank(val_pivot, val_test_user_items, val_comm_user):
    match_indexes = [np.where(pivot.loc[u].isin(set(pivot.loc[u]) & test_users_items[u]))[0] for u in comm_user]
    reciprocal_rank = np.mean([1 / (np.min(index) + 1) if len(index) > 0 else 0 for index in match_indexes])

    return reciprocal_rank

def dcg(val_pivot, val_test_user_items, val_comm_user):
    match_indexes = [np.where(val_pivot.loc[u].isin(set(val_pivot.loc[u]) & val_test_user_items[u]))[0] for u in val_comm_user]
    discounted_cumulative_gain = np.mean([np.sum(1 / np.log2(index + 2)) if len(index) > 0 else 0 for index in match_indexes])
    
    return discounted_cumulative_gain

def precision(val_pivot, val_test_user_items, val_comm_user):
    precision = np.mean([len(set(val_pivot.loc[u]) & val_test_user_items[u]) / len(val_pivot.loc[u]) for u in val_comm_user])

    return precision

def recall(val_pivot, val_test_user_items, val_comm_user):
    recall = np.mean([len(set(val_pivot.loc[u]) & val_test_user_items[u]) / len(test_users_items[u]) for u in val_comm_user])

    return recall

print("EVALUATION CLASSIFICATION TESTSET ONLY\n")

print("hit_rate: {:.3f}".format(hit_rate(pivot, test_users_items, comm_user)))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank(pivot, test_users_items, comm_user)))
print("dcg: {:.3f}".format(dcg(pivot, test_users_items, comm_user)))
print("precision: {:.3f}".format(precision(pivot, test_users_items, comm_user)))
print("recall: {:.3f}".format(recall(pivot, test_users_items, comm_user)))

24/02/09 09:28:18 WARN TaskSetManager: Stage 255 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


EVALUATION CLASSIFICATION TESTSET ONLY

hit_rate: 1.000
reciprocal_rank: 1.000
dcg: 4.544
precision: 1.000
recall: 0.147


#### Prediction on All Combination Data (excluded training data)

In [21]:

spark_dataframe = spark.createDataFrame(df_pns_final)

assembler = VectorAssembler(inputCols=feature_columns_set, outputCol="features")
balanced_df_spark = assembler.transform(spark_dataframe)

(train_data, test_data) = balanced_df_spark.randomSplit([0.75, 0.25], seed=42)

# Ensure 'user_id' is present
print("Training Data Columns: ", train_data.columns)
print("Test Data Columns: ", test_data.columns)

labelIndexer = StringIndexer(inputCol="interaction", outputCol="indexedLabel").fit(balanced_df_spark)

fm = FMClassifier(featuresCol="features", labelCol="indexedLabel", stepSize=0.001)
pipeline = Pipeline(stages=[labelIndexer, fm])
model = pipeline.fit(train_data)

Training Data Columns:  ['user_id', 'item_id', 'interaction', 'gender_F', 'gender_M', 'age_1', 'age_18', 'age_25', 'age_35', 'age_45', 'age_50', 'age_56', 'occupation_0', 'occupation_1', 'occupation_10', 'occupation_11', 'occupation_12', 'occupation_13', 'occupation_14', 'occupation_15', 'occupation_16', 'occupation_17', 'occupation_18', 'occupation_19', 'occupation_2', 'occupation_20', 'occupation_3', 'occupation_4', 'occupation_5', 'occupation_6', 'occupation_7', 'occupation_8', 'occupation_9', 'animation', "children's", 'comedy', 'adventure', 'fantasy', 'romance', 'drama', 'action', 'crime', 'thriller', 'horror', 'sci-fi', 'documentary', 'war', 'musical', 'mystery', 'film-noir', 'western', 'features']
Test Data Columns:  ['user_id', 'item_id', 'interaction', 'gender_F', 'gender_M', 'age_1', 'age_18', 'age_25', 'age_35', 'age_45', 'age_50', 'age_56', 'occupation_0', 'occupation_1', 'occupation_10', 'occupation_11', 'occupation_12', 'occupation_13', 'occupation_14', 'occupation_15', '

24/02/09 09:28:39 WARN TaskSetManager: Stage 258 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:28:40 WARN TaskSetManager: Stage 261 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:28:45 WARN TaskSetManager: Stage 262 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:28:46 WARN TaskSetManager: Stage 264 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:28:47 WARN TaskSetManager: Stage 266 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:28:48 WARN TaskSetManager: Stage 268 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:28:48 WARN TaskSetManager: Stage 270 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.

In [22]:
predictions = model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="interaction", rawPredictionCol="prediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f"Area under ROC: {roc_auc}")

evaluator = BinaryClassificationEvaluator(labelCol="interaction")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

evaluator = MulticlassClassificationEvaluator(labelCol="interaction", predictionCol="prediction")

precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

print("Weighted Precision: {:.3f}".format(precision))
print("Weighted Recall: {:.3f}".format(recall))

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

24/02/09 09:33:52 WARN TaskSetManager: Stage 464 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


Area under ROC: 0.49676549889536636


24/02/09 09:34:02 WARN TaskSetManager: Stage 475 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


Accuracy: 0.491118937378115


24/02/09 09:34:30 WARN TaskSetManager: Stage 486 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:34:43 WARN TaskSetManager: Stage 488 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


Weighted Precision: 0.444
Weighted Recall: 0.461


24/02/09 09:34:56 WARN TaskSetManager: Stage 490 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         0.0|(50,[0,1,2,4,13,3...|
|       1.0|         0.0|(50,[0,1,2,4,13,3...|
|       1.0|         0.0|(50,[0,1,2,4,13,3...|
|       1.0|         0.0|(50,[0,1,2,4,13,3...|
|       1.0|         0.0|(50,[0,1,2,4,13,3...|
+----------+------------+--------------------+
only showing top 5 rows



In [23]:
from sklearn.metrics import confusion_matrix

y_pred = predictions.select("prediction").collect()
y_orig = predictions.select("interaction").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix: \n", cm)

24/02/09 09:34:57 WARN TaskSetManager: Stage 491 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:35:46 WARN TaskSetManager: Stage 492 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


Confusion Matrix: 
 [[211755   3977]
 [246804   2989]]


In [24]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# # Rank the predictions
windowSpec = Window.partitionBy("user_id").orderBy(predictions["prediction"].desc())
df_test = predictions.withColumn("rank", row_number().over(windowSpec))

# Filter to get top 10 predictions for each user
top_10_recommendations = df_test.filter(df_test['rank'] <= 10)

# Pivot the data to get a wide format DataFrame with one row per user and top 10 movie recommendations
pivot_recommendations = top_10_recommendations.groupBy("user_id").pivot("rank").agg({"item_id": "first"})
pivot_recommendations = pivot_recommendations.na.fill(0)
pivot_recommendations.show()


24/02/09 09:36:01 WARN TaskSetManager: Stage 493 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.
24/02/09 09:36:15 WARN TaskSetManager: Stage 506 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


+-------+----+----+----+----+----+----+----+----+----+----+
|user_id|   1|   2|   3|   4|   5|   6|   7|   8|   9|  10|
+-------+----+----+----+----+----+----+----+----+----+----+
|      7|1196|3107|3753|2302|2968|2291| 334| 588| 608|2000|
|     19|  34|  76| 223| 368| 377| 648| 785|1090|1127|1219|
|     22| 648|  47|  81|  95| 104| 163| 180| 256| 312| 333|
|     25|1964| 110| 157| 223| 260| 546| 737|1129|1356|1371|
|     26|   1|  39|  45| 104| 125| 160| 168| 195| 198| 234|
|     29|  50| 288| 318| 589| 858| 912| 969|1225|1356|1374|
|     31| 951|1235|1259|1304|1485|2300|2371|2423|2716|2747|
|     32| 457| 608|1343|1683|2916|2959|3079|3786|3897|1267|
|     34| 339| 353| 441| 455| 497| 837|1079|1100|1148|1196|
|     39| 246| 903| 350| 223| 785|1060|1127|1193|2605|2706|
|     43|1625|2369|2581|2605|2961|3298|1196|1199|3698|  39|
|     50|1909| 581|2241|3155|3409|3565|3594|3786|1059| 919|
|     54|1269|3176| 909|1193|1235|2788|1246|1961|2355|1653|
|     57|  70| 441| 661|1196|1199|1221|1

In [25]:
pivot = pivot_recommendations.toPandas().set_index('user_id')
pivot = pivot.rename_axis(None, axis=0)
pivot

24/02/09 09:36:28 WARN TaskSetManager: Stage 509 contains a task of very large size (1687 KiB). The maximum recommended task size is 1000 KiB.


,1,2,3,4,5,6,7,8,9,10
7,1196,3107,3753,2302,2968,2291,334,588,608,2000
19,34,76,223,368,377,648,785,1090,1127,1219
22,648,47,81,95,104,163,180,256,312,333
25,1964,110,157,223,260,546,737,1129,1356,1371
26,1,39,45,104,125,160,168,195,198,234
...,...,...,...,...,...,...,...,...,...,...
6038,61,232,920,1079,1183,1210,1419,3088,1466,3481
6039,199,471,661,858,900,901,904,912,916,924
5703,303,541,1263,1304,1340,1345,1617,2401,2633,2951
5744,3,7,10,11,17,31,39,48,150,165


In [34]:

test_users_items = df_test.toPandas().groupby('user_id')['item_id'].apply(set).to_dict()
comm_user = pivot.index.values


print("EVALUATION CLASSIFICATION TESTSET ONLY\n")

print("hit_rate: {:.3f}".format(hit_rate(pivot, test_users_items, comm_user)))
print("reciprocal_rank: {:.3f}".format(reciprocal_rank(pivot, test_users_items, comm_user)))
print("dcg: {:.3f}".format(dcg(pivot, test_users_items, comm_user)))
print("precision: {:.3f}".format(precision(pivot, test_users_items, comm_user)))
print("recall: {:.3f}".format(recall(pivot, test_users_items, comm_user)))

EVALUATION CLASSIFICATION TESTSET ONLY

hit_rate: 1.000
reciprocal_rank: 1.000
dcg: 4.544
precision: 0.999
recall: 0.158


**Notes:** 

- Its significant lower when predicting over unseen data and back testing into test data

#### **Result Comparison**

| Metrics | FMClassifier (Randomized Negative Sampling) | FMClassifier (Popular-based Negative Sampling) | RankFM |
| --- | --- | --- | --- | 
| hit_rate | 1.000 |  1.000 | 0.788 |
| reciprocal_rank | 1.000 | 1.000 | 0.334 |
| dcg | 4.544 | 4.544 | 0.718 |
| precision | 1.000 | 0.999 | 0.156 |
| recall | 0.147 | 0.158 | 0.072 |
